In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ase
from ase import Atoms, Atom

import tensorflow as tf
from tensorflow.keras import layers

from Constants import Keys
from prepare_data import (
    create_grid, 
    deserialize_df, 
    filter_out_single_atom_molecules,
    create_matrices,
    create_df
)

In [2]:
df0 = pd.read_pickle("./../Datasets/Cu_df2_1k.pkl.gzip", compression="gzip")
atoms = df0["ase_atoms"]
a = Atoms(atoms[800])
print(a.positions)
df0

[[ 0.        0.        0.      ]
 [ 1.613117  1.613117  1.613117]
 [ 0.        0.        3.226233]
 [ 1.613117  1.613117  4.83935 ]
 [ 0.        0.        6.452467]
 [ 1.613117  1.613117  8.065583]
 [ 0.        0.        9.6787  ]
 [ 1.613117  1.613117 11.291817]]


,name,ase_atoms,energy_corrected,forces,energy_corrected_per_atom
0,A1:FHI-aims/PBE/tight:elastic:s_e_0,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",-3.699843,"[[0.0, 0.0, 0.0]]",-3.699843
1,A1:FHI-aims/PBE/tight:murnaghan:strain_1_0,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",-3.699841,"[[0.0, 0.0, 0.0]]",-3.699841
2,A1:FHI-aims/PBE/tight:phonon:supercell_phonon_0,"(Atom('Cu', [4.3368086899420173e-19, 0.0070709...",-236.789603,"[[-1.13852957740976e-06, -0.0464638907314277, ...",-3.699838
3,A1:FHI-aims/PBE/tight:murnaghan:strain_1_02,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",-3.697932,"[[0.0, 0.0, 0.0]]",-3.697932
4,A1:FHI-aims/PBE/tight:qha:phonopy_strain_1_02:...,"(Atom('Cu', [0.0, 0.007070999999999999, 0.0070...",-236.667372,"[[-1.7950713845026e-06, -0.0426786586259085, -...",-3.697928
...,...,...,...,...,...
995,Zr4Al3,"(Atom('Cu', [1.397, 0.806558, 0.0], index=0), ...",-18.922813,"[[-6.18422096975777e-08, 3.90650715661433e-08,...",-2.703259
996,fcc_100surf_X110_Y-110_Z001_4at_d0.95,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-10.811770,"[[-4.30878346990838e-07, -7.31495829958533e-08...",-2.702942
997,ran2.7197,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-5.405739,"[[-0.233777024216308, -0.488775610406501, 0.45...",-2.702869
998,A15.shakesmallsuper2.14,"(Atom('Cu', [19.532944999999994, -0.449866, 19...",-172.952480,"[[0.872967501214253, 0.798687481685555, 0.8365...",-2.702382


In [3]:
df0 = deserialize_df(df0)
grid = create_grid(df0["ase_atoms"][800], 1)
grid

[GridCell(index: (0, 0, 0), atoms: Atom('Cu', [0.0, 0.0, 0.0], index=0)),
 GridCell(index: (1, 1, 1), atoms: Atom('Cu', [1.613117, 1.613117, 1.613117], index=1)),
 GridCell(index: (0, 0, 3), atoms: Atom('Cu', [0.0, 0.0, 3.2262330000000006], index=2)),
 GridCell(index: (1, 1, 4), atoms: Atom('Cu', [1.613117, 1.613117, 4.8393500000000005], index=3)),
 GridCell(index: (0, 0, 6), atoms: Atom('Cu', [0.0, 0.0, 6.452467], index=4)),
 GridCell(index: (1, 1, 8), atoms: Atom('Cu', [1.613117, 1.613117, 8.065583], index=5)),
 GridCell(index: (0, 0, 9), atoms: Atom('Cu', [0.0, 0.0, 9.678700000000001], index=6)),
 GridCell(index: (1, 1, 11), atoms: Atom('Cu', [1.613117, 1.613117, 11.291817], index=7))]

In [4]:
create_df("./../Datasets/Cu_df2_1k.pkl.gzip", grid_size=0.1, save_path="./../Datasets/Cu_df_preprocessed")

,name,ase_atoms,energy_corrected,forces,energy_corrected_per_atom,grid,local_distance_matrix,local_atomic_numbers,long_range_distance_matrix,long_range_atomic_features,n_max_local,n_max_long_range
0,A1:FHI-aims/PBE/tight:phonon:supercell_phonon_0,"(Atom('Cu', [4.3368086899420173e-19, 0.0070709...",-236.789603,"[[-1.13852957740976e-06, -0.0464638907314277, ...",-3.699838,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[2.5576886661519227, -1.300522737268895e-16,...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,63
1,A1:FHI-aims/PBE/tight:qha:phonopy_strain_1_02:...,"(Atom('Cu', [0.0, 0.007070999999999999, 0.0070...",-236.667372,"[[-1.7950713845026e-06, -0.0426786586259085, -...",-3.697928,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[2.5743339597810535, 0.0, -0.707106781186547...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,63
2,fcc_100surf_X110_Y-110_Z001_2at_d1.05,"(Atom('Cu', [1.2699999999999998, 1.26999999999...",-7.393033,"[[1.43567879745929e-07, 1.29341091704393e-08, ...",-3.696516,"[GridCell(index: (12, 12, 0), atoms: Atom('Cu'...",[],[],"[[[2.5399998414568845, 0.5000000312092766, 0.5...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1
3,A1:FHI-aims/PBE/tight:tp:trigonal:tp_4_12500,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-7.391491,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]",-3.695746,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[6.419884, 0.0, -3.4587011996639086e-17, -1....","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1
4,fcc_110surf_X100_Y01-1_Z011_2at_d1.00,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-7.391475,"[[-9.87788653663608e-08, -6.83294122864741e-08...",-3.695738,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[2.590799824143888, -0.70710673319015, -0.50...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
833,Zr4Al3,"(Atom('Cu', [1.397, 0.806558, 0.0], index=0), ...",-18.922813,"[[-6.18422096975777e-08, 3.90650715661433e-08,...",-2.703259,"[GridCell(index: (13, 8, 0), atoms: Atom('Cu',...",[],[],"[[[2.794000018902112, -0.49999999661737443, -0...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,6
834,fcc_100surf_X110_Y-110_Z001_4at_d0.95,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-10.811770,"[[-4.30878346990838e-07, -7.31495829958533e-08...",-2.702942,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[2.2859999280218712, -0.5000000157432483, -0...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
835,ran2.7197,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-5.405739,"[[-0.233777024216308, -0.488775610406501, 0.45...",-2.702869,"[GridCell(index: (8, 0, 0), atoms: Atom('Cu', ...",[],[],"[[[10.07632976392466, 0.08185043754252488, -0....","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1
836,A15.shakesmallsuper2.14,"(Atom('Cu', [19.532944999999994, -0.449866, 19...",-172.952480,"[[0.872967501214253, 0.798687481685555, 0.8365...",-2.702382,"[GridCell(index: (195, 0, 192), atoms: Atom('C...",[],[],"[[[24.063802685741834, 0.7104031820427551, -0....","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,63


In [5]:
def create_tf_dataset(path, batch_size=32, shuffle_buffer_size=1000):
    df = pd.read_pickle(path, compression="gzip")
    
    local_distance_matrices = np.stack(np.array(df[Keys.LOCAL_DISTANCE_MATRIX_KEY]))
    local_atomic_numbers = np.stack(np.array(df[Keys.LOCAL_ATOMIC_NUMBERS_KEY]))
    long_range_distance_matrices = np.stack(np.array(df[Keys.LONG_RANGE_DISTANCE_MATRIX_KEY]))
    long_range_atomic_features = np.stack(np.array(df[Keys.LONG_RANGE_ATOMIC_FEATURES_KEY]))
    energies = np.stack(np.array(df[Keys.ENERGY_KEY]))
    forces = np.stack(np.array(df[Keys.FORCE_KEY]))
    
    print(local_distance_matrices.shape)
    print(local_atomic_numbers.shape)
    print(long_range_distance_matrices.shape)
    print(long_range_atomic_features.shape)
    print(energies.shape)
    print(forces.shape)
    
    # create tf dataset
    dataset = tf.data.Dataset.from_tensor_slices(
        (
            local_distance_matrices,
            local_atomic_numbers,
            long_range_distance_matrices,
            long_range_atomic_features,
            energies,
            forces
        )
    )
    
    # shuffle and batch
    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    
    return dataset

create_tf_dataset("./../Datasets/Cu_df_preprocessed_grid_size_0.1.pkl.gzip", batch_size=32, shuffle_buffer_size=1000)

ValueError: all input arrays must have the same shape